## 방문지 Data Cleansing

- 결측치 및 이상치 탐색

In [2]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
font_path = r'C:\Windows\Fonts\HMFMMUEX.TTC'
font_name = fm.FontProperties(fname=font_path).get_name()
plt.rc('font', family=font_name)
plt.rcParams['axes.unicode_minus'] = False

#### 데이터 확인

In [3]:
visit_info = pd.read_csv('data/tn_visit_area_info_방문지정보_E.csv')

In [4]:
visit_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21384 entries, 0 to 21383
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   VISIT_AREA_ID        21384 non-null  int64  
 1   TRAVEL_ID            21384 non-null  object 
 2   VISIT_ORDER          21384 non-null  int64  
 3   VISIT_AREA_NM        21384 non-null  object 
 4   VISIT_START_YMD      21384 non-null  object 
 5   VISIT_END_YMD        21384 non-null  object 
 6   ROAD_NM_ADDR         18293 non-null  object 
 7   LOTNO_ADDR           16044 non-null  object 
 8   X_COORD              14486 non-null  float64
 9   Y_COORD              14486 non-null  float64
 10  ROAD_NM_CD           1338 non-null   float64
 11  LOTNO_CD             1338 non-null   float64
 12  POI_ID               14674 non-null  object 
 13  POI_NM               14674 non-null  object 
 14  RESIDENCE_TIME_MIN   18821 non-null  float64
 15  VISIT_AREA_TYPE_CD   21384 non-null 

In [53]:
len(visit_info)

21384

In [5]:
visit_info.isnull().sum()

VISIT_AREA_ID              0
TRAVEL_ID                  0
VISIT_ORDER                0
VISIT_AREA_NM              0
VISIT_START_YMD            0
VISIT_END_YMD              0
ROAD_NM_ADDR            3091
LOTNO_ADDR              5340
X_COORD                 6898
Y_COORD                 6898
ROAD_NM_CD             20046
LOTNO_CD               20046
POI_ID                  6710
POI_NM                  6710
RESIDENCE_TIME_MIN      2563
VISIT_AREA_TYPE_CD         0
REVISIT_YN              5651
VISIT_CHC_REASON_CD     5627
LODGING_TYPE_CD        20216
DGSTFN                  5659
REVISIT_INTENTION       5659
RCMDTN_INTENTION        5659
SGG_CD                 15993
dtype: int64

In [ ]:
# pd.set_option('display.max_columns', None)
# temp = visit_info.head()
# temp.to_csv('visit_info_head.csv', encoding='utf-8-sig')

#### 1 ~ 6 여행, 방문지, 방문순서, 방문 시작일, 종료일

- 방문지역에는 다수의 여행객이 매핑

In [ ]:
temp = visit_info[['VISIT_AREA_ID', 'TRAVEL_ID', 'VISIT_ORDER', 'VISIT_AREA_NM', 'VISIT_START_YMD', 'VISIT_END_YMD']]    
temp[temp['VISIT_AREA_ID'] == 2304300002] 

,VISIT_AREA_ID,TRAVEL_ID,VISIT_ORDER,VISIT_AREA_NM,VISIT_START_YMD,VISIT_END_YMD
1,2304300002,e_e000004,2,화성 관광열차 안내소 연무대 매표소,2023-04-30,2023-04-30
6,2304300002,e_e000006,2,청량리역 경춘선,2023-04-30,2023-04-30
23,2304300002,e_e000009,8,샤오바오 고덕점,2023-04-30,2023-04-30
39,2304300002,e_e000010,12,미사리밀빛초계국수 롯데월드점,2023-04-30,2023-04-30
53,2304300002,e_e000011,11,동원 홈 푸드라운지 국립 현대미술관점,2023-04-30,2023-04-30
78,2304300002,e_e000015,11,에인절 코인 노래연습장 건대점,2023-04-30,2023-04-30
108,2304300002,e_e000020,9,가평 양떼목장,2023-04-30,2023-04-30
123,2304300002,e_e000021,11,티 카페 차희,2023-04-30,2023-04-30
142,2304300002,e_e000025,5,집,2023-04-30,2023-05-01
152,2304300002,e_e000037,4,강촌 아파트,2023-04-30,2023-05-01


- 여행id 기준으로 확인
    - 여행 (순서)여정 확인 가능 : TRAVEL_ID 기준 로우 카운트
    - 총 여행 소요일 확인 가능 : VISIT_END_YMD - VISIT_START_YMD

In [ ]:
temp = visit_info[['VISIT_AREA_ID', 'TRAVEL_ID', 'VISIT_ORDER', 'VISIT_AREA_NM', 'VISIT_START_YMD', 'VISIT_END_YMD']]    
temp = temp[temp['TRAVEL_ID']=='e_e000004'] 

In [41]:
temp

,VISIT_AREA_ID,TRAVEL_ID,VISIT_ORDER,VISIT_AREA_NM,VISIT_START_YMD,VISIT_END_YMD
0,2304300001,e_e000004,1,집,2023-04-30,2023-04-30
1,2304300002,e_e000004,2,화성 관광열차 안내소 연무대 매표소,2023-04-30,2023-04-30
2,2304300003,e_e000004,3,창룡문,2023-04-30,2023-04-30
3,2304300004,e_e000004,4,수원 화성 화홍문,2023-04-30,2023-04-30
4,2304300005,e_e000004,5,집,2023-04-30,2023-05-01


- 여행 id에 어떤 규칙이 있는지 확인해보기 

In [ ]:
# temp = visit_info[['VISIT_AREA_ID', 'TRAVEL_ID', 'VISIT_ORDER', 'VISIT_AREA_NM', 'VISIT_START_YMD', 'VISIT_END_YMD']]    
# temp = temp[temp['TRAVEL_ID']=='f_f000038'] 

#### 7 ~ 12 방문지 주소 관련 컬럼

In [ ]:
# ROAD_NM_ADDR	도로명주소
# LOTNO_ADDR	지번주소
# X_COORD	    X좌표
# Y_COORD	    Y좌표
# ROAD_NM_CD	도로명코드
# LOTNO_CD	    지번코드

- 각 주소는 방문지마다의 주소로 도로명 / 지번 / 위도, 경도 정보가 있음 
- 방문지 마다의 주소가 필요한지 확인 (여행 지역만 본다하면 주소컬럼 제외)
- 필요하면 하나의 컬럼으로 정리하자
- 기준은 도로명주소로 통일

In [ ]:
temp = visit_info[['VISIT_AREA_ID', 'TRAVEL_ID', 'VISIT_ORDER', 'VISIT_AREA_NM', 'ROAD_NM_ADDR', 'LOTNO_ADDR', 'X_COORD' ,'Y_COORD', 'ROAD_NM_CD', 'LOTNO_CD']]    

travel_id = 'f_f000038' # 'e_e000004'
temp = temp[temp['TRAVEL_ID']==travel_id] 
temp

,VISIT_AREA_ID,TRAVEL_ID,VISIT_ORDER,VISIT_AREA_NM,ROAD_NM_ADDR,LOTNO_ADDR,X_COORD,Y_COORD,ROAD_NM_CD,LOTNO_CD
21287,2304290001,f_f000038,1,숙소,영동대로 726,NaN,NaN,NaN,NaN,NaN
21288,2304290002,f_f000038,5,친척 집,엑스포아파트,NaN,NaN,NaN,NaN,NaN
21289,2304290003,f_f000038,2,친척 집,NaN,NaN,NaN,NaN,NaN,NaN
21290,2304290004,f_f000038,3,박통,NaN,NaN,NaN,NaN,NaN,NaN
21291,2304290005,f_f000038,4,김경자 소문난 대구왕뽈찜 서구점,대전 서구 월평북로 91,대전 서구 월평동 240,127.378574,36.362488,NaN,NaN
21292,2304300001,f_f000038,11,동탄 호수 자이 파밀리에 아파트,경기 화성시 동탄대로2길 19,경기 화성시 장지동 963,127.107081,37.159923,NaN,NaN
21293,2304300002,f_f000038,6,맥도날드,NaN,NaN,NaN,NaN,NaN,NaN
21294,2304300003,f_f000038,7,롯데백화점 동탄점,경기 화성시 동탄역로 160,경기 화성시 오산동 967-2419,127.097919,37.200709,NaN,NaN
21295,2304300004,f_f000038,8,동탄 호수 공원,NaN,경기 화성시 송동 54-1,127.104522,37.167834,NaN,NaN
21296,2304300005,f_f000038,9,명륜 진사갈비 화성 남동탄점,경기 화성시 동탄장지천5길 6,경기 화성시 장지동 962-6,127.106697,37.158200,NaN,NaN


- ROAD_NM_CD, LOTNO_CD 전체중 94%가 널이기도 하고 중복 정보로 제외 	

In [71]:
temp = visit_info[['VISIT_AREA_ID', 'TRAVEL_ID', 'VISIT_ORDER', 'VISIT_AREA_NM', 'ROAD_NM_ADDR', 'LOTNO_ADDR', 'X_COORD' ,'Y_COORD', 'ROAD_NM_CD', 'LOTNO_CD']]    

print(temp['ROAD_NM_CD'].isnull().sum() / len(temp))
print(temp['LOTNO_CD'].isnull().sum() / len(temp))

0.9374298540965208
0.9374298540965208


#### 13, 14 poi master 사용안하므로 제외

In [4]:
temp = visit_info[['VISIT_AREA_ID', 'TRAVEL_ID', 'VISIT_ORDER', 'VISIT_AREA_NM', 'POI_ID', 'POI_NM']]    
temp

,VISIT_AREA_ID,TRAVEL_ID,VISIT_ORDER,VISIT_AREA_NM,POI_ID,POI_NM
0,2304300001,e_e000004,1,집,NaN,NaN
1,2304300002,e_e000004,2,화성 관광열차 안내소 연무대 매표소,POI01000000ALZU7R,동대문종합시장 악세서리부자재시장
2,2304300003,e_e000004,3,창룡문,POI010000006N1USC,창룡문
3,2304300004,e_e000004,4,수원 화성 화홍문,POI01000TR021821V,수원화성 화홍문
4,2304300005,e_e000004,5,집,NaN,NaN
...,...,...,...,...,...,...
21379,2308270002,h_h003275,2,용인농촌테마파크,POI010000006S1184,용인농촌테마파크
21380,2308270003,h_h003275,3,내동마을 연꽃 단지,POI03000TR018341V,내동마을연꽃단지
21381,2308270004,h_h003275,4,외할머니 집,POI01000000CT5Q2Y,외할머니집
21382,2308270005,h_h003275,5,삼성화재 모빌리티 뮤지엄,POI03000TR018343V,삼성화재 모빌리티뮤지엄


#### 15 체류시간(분)

In [ ]:
# RESIDENCE_TIME_MIN

#### 16 ~ 22 방문지, 숙소 만족도 관련 

In [ ]:
# VISIT_AREA_TYPE_CD        방문지유형코드
# REVISIT_YN                재방문여부
# VISIT_CHC_REASON_CD       방문선택이유코드
# LODGING_TYPE_CD           숙소유형코드
# DGSTFN                    만족도
# REVISIT_INTENTION         재방문의향
# RCMDTN_INTENTION          추천의향

#### 23 시군구코드

In [ ]:
# SGG_CD